In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import re
import os
import pickle
import warnings
import wget
import sklearn
import random
import nltk
import spacy
import textblob
import gensim
import Sastrawi
# import sparknlp
# from sparknlp.base import DocumentAssembler
from gensim.models import Word2Vec
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, GRU, SimpleRNN, Conv1D, MaxPooling1D, Flatten
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
# from keras.utils.data_utils import pad_sequences
from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, recall_score, precision_score
from nltk.corpus import stopwords
from nltk.tree import Tree
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from collections import Counter, OrderedDict
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from spacy.util import minibatch, compounding
from spacy import load, displacy
from spacy.training.example import Example 
from nlp_id import stopword
from unidecode import unidecode
import io
import time
from datetime import timedelta
import logging
from gensim.models import FastText
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras import backend as K
# from keras.optimizers.legacy import Adam
# from sparknlp.annotator import ContextSpellCheckerModel, NorvigSweetingModel, SymmetricDeleteModel
logging.basicConfig(level=logging.INFO)
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hammam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hammam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
data = pd.read_csv('./normalized_tweet.csv')

In [8]:
token_data = []

for index in range(len(data.tweet)):
    tokens = word_tokenize(data.tweet[index])
    token_data.append(tokens)

token_data

[['kata',
  'prabowo',
  'indonesia',
  'tidak',
  'harga',
  'bangsa',
  'asing',
  'berita',
  'hoax',
  'buat',
  'kuasa',
  'ya',
  'kan',
  'rockygerung'],
 ['batu',
  'langka',
  'tasbih',
  'jokowi',
  'hadiah',
  'habib',
  'luthfi',
  'harga',
  'mercy'],
 ['era', 'jokowi', 'ekonomi', 'indonesia', 'makin', 'baik'],
 ['sumatera',
  'selatan',
  'asi',
  'games',
  'dampak',
  'pd',
  'ekonomi',
  'langsung',
  'prediksi',
  'capai',
  '185',
  'triliun',
  'indonesia',
  'maju',
  'jokowi',
  'hebat'],
 ['negara',
  'ngutang',
  'buat',
  'bngun',
  'infrastruktur',
  'udah',
  'dipake',
  'masyarakat',
  'terus',
  'masyarakat',
  'ngeluh',
  'negara',
  'ngutang',
  'negara',
  'ngutang',
  'utang',
  'bayar',
  'kalo',
  'negara',
  'dapet',
  'hasil',
  'hasil',
  'negara',
  'ya',
  'pajak'],
 ['yg',
  'bisikin',
  'pak',
  'jokowi',
  'cm',
  'mikirin',
  'putar',
  'duit',
  'golong',
  'tentu',
  'esports',
  'bagi',
  'kecil',
  'dr',
  'sekian',
  'besar',
  'tentu',


In [10]:
len(token_data)

1815

In [11]:
total = 0
for token in token_data:
    total += len(token)

print(total)

35372


In [12]:
total * 405898

14357424056

In [13]:
normalized_data = pd.read_csv('./normalized_tweet.csv')
data = pd.read_csv('./tweet.csv')
dict_id = './processed_uncased_blanklines/wiki.txt'

In [14]:
display(normalized_data)
display(data)
display(dict_id)

,Unnamed: 0,tweet
0,0,kata prabowo indonesia tidak harga bangsa asin...
1,1,batu langka tasbih jokowi hadiah habib luthfi ...
2,2,era jokowi ekonomi indonesia makin baik
3,3,sumatera selatan asi games dampak pd ekonomi l...
4,4,negara ngutang buat bngun infrastruktur udah d...
...,...,...
1810,1810,negarawan sejati sll bangga depan harga diri b...
1811,1811,1 hrs ceramah damai indonesia 2 ekonomi makin ...
1812,1812,bangun bangsa dgn dukung ekonomi negara sama p...
1813,1813,bantu maju ekonomi bangsa sama pak jokowi yuk


,Unnamed: 0,sentimen,tweet
0,0,negatif,Kata @prabowo Indonesia tidak dihargai bangsa ...
1,1,netral,"Batuan Langka, Tasbih Jokowi Hadiah dari Habib..."
2,2,netral,"Di era Jokowi, ekonomi Indonesia semakin baik...."
3,3,positif,"Bagi Sumatera Selatan, Asian Games berdampak p..."
4,4,negatif,Negara kita ngutang buat bngun infrastruktur y...
...,...,...,...
1810,1810,netral,Negarawan sejati sll bangga dan mengedepankan ...
1811,1811,netral,1. HRS ceramah di Damai Indonesiaku 2. Perekon...
1812,1812,netral,Mari bangun bangsa dgn mendukung perekonomian ...
1813,1813,netral,Bantu majukan perekonomian bangsa bersama Pak ...


'./processed_uncased_blanklines/wiki.txt'

# Spell Checking

In [19]:
def create_spell_check_dictionary(spell_check_data):
    spell_check_dict = {}
    lines = spell_check_data.strip().split("\n")
    for line in lines:
        key, value = line.split(":")
        spell_check_dict[key] = value
    return spell_check_dict

In [41]:
spell_dictionary = open('./slangword.txt').read()
spell_dictionary

'&:dan\n+:tambah\n/:atau\n=:sama dengan\nababil:anak ingusan\nabal2:palsu\nabal:palsu\nad:ada\nakooh:aku\nalay:norak\nalbm:album\nampe:sampai\nanjir:waw\nanyway:ngomong-ngomong\naq:aku\nasap:secepatnya\nato:atau\natw:atau\nava:foto profil\nbaget:keras kepala\nbaper:bawa perasaan\nbapuk:rusak\nbcr:bicara\nbebeb:pacar\nbegin:awal\nbejibun:bertumpuk banyak\nbener:benar\nber2:berdua\nber3:bertiga\nbetter:lebih baik\nbf:pacar\nbgt:banget\nbhas:bahas\nbhg:bahagia\nbikin:buat\nbinggo:banget\nbingit:banget\nbklan:bakalan\nbkn:bukan\nblg:bilang\nbnr:benar\nbnyk:banyak\nbr:baru\nbrb:akan segera kembali\nbrjaya:berjaya\nbs:bisa\nbsk:besok\nbtw:ngomong-ngomong\ncm:cuma\ncmiiw:koreksi saya\ncnt:cinta\ncongrats:selamat\ncrop:potong\ncya:sampai jumpa lagi\nd:di\nda:ada\ndafuk:kurang ajar\ndamat:tidak peduli\ndempa:demi apa\ndgn:dengan\ndkt:dekat\ndlm:dalam\ndlu:dulu\ndmn:dimana\ndpt:dapat\ndr:dari\ndrpd:daripada\ndsna:disana\ndugem:dunia gemerlap\nelo:kamu\nelu:kamu\nemg:memang\nepsd:episode\nfake:fa

In [42]:
spell_dictionary = create_spell_check_dictionary(spell_dictionary)
spell_dictionary

{'&': 'dan',
 '+': 'tambah',
 '/': 'atau',
 '=': 'sama dengan',
 'ababil': 'anak ingusan',
 'abal2': 'palsu',
 'abal': 'palsu',
 'ad': 'ada',
 'akooh': 'aku',
 'alay': 'norak',
 'albm': 'album',
 'ampe': 'sampai',
 'anjir': 'waw',
 'anyway': 'ngomong-ngomong',
 'aq': 'aku',
 'asap': 'secepatnya',
 'ato': 'atau',
 'atw': 'atau',
 'ava': 'foto profil',
 'baget': 'keras kepala',
 'baper': 'bawa perasaan',
 'bapuk': 'rusak',
 'bcr': 'bicara',
 'bebeb': 'pacar',
 'begin': 'awal',
 'bejibun': 'bertumpuk banyak',
 'bener': 'benar',
 'ber2': 'berdua',
 'ber3': 'bertiga',
 'better': 'lebih baik',
 'bf': 'pacar',
 'bgt': 'banget',
 'bhas': 'bahas',
 'bhg': 'bahagia',
 'bikin': 'buat',
 'binggo': 'banget',
 'bingit': 'banget',
 'bklan': 'bakalan',
 'bkn': 'bukan',
 'blg': 'bilang',
 'bnr': 'benar',
 'bnyk': 'banyak',
 'br': 'baru',
 'brb': 'akan segera kembali',
 'brjaya': 'berjaya',
 'bs': 'bisa',
 'bsk': 'besok',
 'btw': 'ngomong-ngomong',
 'cm': 'cuma',
 'cmiiw': 'koreksi saya',
 'cnt': 'cinta

In [43]:
def tweet_spell_check(tweet):
    # Split tweet into words
    words = tweet.split()
    
    # Initialize a list to store corrected words
    corrected_words = []
    
    # Iterate through words in the tweet
    for word in words:
        # Check if the word is in the spell check dictionary
        if word in spell_dictionary:
            corrected_word = spell_dictionary[word]
            corrected_words.append(corrected_word)
        else:
            corrected_words.append(word)  # Keep the word as is if not found in the dictionary
    
    # Join the corrected words back into a tweet
    corrected_tweet = ' '.join(corrected_words)
    
    return corrected_tweet

In [44]:
tweet_spell_corrected = []

for idx in range(len(normalized_data['tweet'])):
    corrected = tweet_spell_check(normalized_data['tweet'][idx])
    tweet_spell_corrected.append(corrected)

display(normalized_data['tweet'][248])
display(tweet_spell_corrected[248])

'pikir kembang esport main doang aplikasi yg smartphone byk yg mulai dr main doang terus jd aplikasi2 yg byk manfaat kata siapa anggur gaji byk baca buka wawas jgn mpermalukan gen z'

'pikir kembang esport main doang aplikasi yang smartphone banyak yang mulai dari main doang terus jadi aplikasi2 yang banyak manfaat kata siapa anggur gaji banyak baca buka wawas jangan mpermalukan generasi z'

## Remove Stopwords

In [45]:
raw_stopwords = ['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua', 'ia', 'seperti', 'jika', 'jika', 'sehingga', 'kembali', 'dan', 'ini', 'karena', 'kepada', 'oleh', 'saat', 'harus', 'sementara', 'setelah', 'belum', 'kami', 'sekitar', 'bagi', 'serta', 'di', 'dari', 'telah', 'sebagai', 'masih', 'hal', 'ketika', 'adalah', 'itu', 'dalam', 'bisa', 'bahwa', 'atau', 'hanya', 'kita', 'dengan', 'akan', 'juga', 'ada', 'mereka', 'sudah', 'saya', 'terhadap', 'secara', 'agar', 'lain', 'anda', 'begitu', 'mengapa', 'kenapa', 'yaitu', 'yakni', 'daripada', 'itulah', 'lagi', 'maka', 'tentang', 'demi', 'dimana', 'kemana', 'pula', 'sambil', 'sebelum', 'sesudah', 'supaya', 'guna', 'kah', 'pun', 'sampai', 'sedangkan', 'selagi', 'sementara', 'tetapi', 'apakah', 'kecuali', 'sebab', 'selain', 'seolah', 'seraya', 'seterusnya', 'tanpa', 'agak', 'boleh', 'dapat', 'dsb', 'dst', 'dll', 'dahulu', 'dulunya', 'anu', 'demikian', 'tapi', 'ingin', 'juga', 'nggak', 'mari', 'nanti', 'melainkan', 'oh', 'ok', 'seharusnya', 'sebetulnya', 'setiap', 'setidaknya', 'sesuatu', 'pasti', 'saja', 'toh', 'walau', 'tolong', 'tentu', 'amat', 'apalagi', 'bagaimanapun']

In [46]:
def remove_stopwords_indo(text):
    # Kode untuk menghapus stopwords
    text_lower = text.lower()
    stop_words = set(raw_stopwords)
    tokens = word_tokenize(text_lower)
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    cleaned_text = ' '.join(filtered_tokens)
    return cleaned_text

In [47]:
tweet_no_stopwords = []

for idx in range(len(tweet_spell_corrected)):
    cleaned = remove_stopwords_indo(tweet_spell_corrected[idx])
    tweet_no_stopwords.append(cleaned)

display(tweet_spell_corrected[248])
display(tweet_no_stopwords[248])

'pikir kembang esport main doang aplikasi yang smartphone banyak yang mulai dari main doang terus jadi aplikasi2 yang banyak manfaat kata siapa anggur gaji banyak baca buka wawas jangan mpermalukan generasi z'

'pikir kembang esport main doang aplikasi smartphone banyak mulai main doang terus jadi aplikasi2 banyak manfaat kata siapa anggur gaji banyak baca buka wawas jangan mpermalukan generasi z'